In [1]:
import importlib
import pandas as pd
import numpy as np
import os
import py_vollib_vectorized
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

In [2]:
import classes, utils, iv_models
_ = importlib.reload(classes)
_ = importlib.reload(utils)
_ = importlib.reload(iv_models)

from classes import Ticker
from utils import create_market_state
from iv_models import BasicMidIVPolynomial, MidIV

In [3]:
t = Ticker("RELIANCE", expiry_date="20210826")
oc1 = t.get_option_chain('20210805 15:15:00')
oc2 = t.get_option_chain('20210806 10:30:00')
oc1.fit_iv_model(BasicMidIVPolynomial(2))
oc2.fit_iv_model(BasicMidIVPolynomial(2))

globbing ./data/*/*_RELIANCE21AUG*.csv
loading data from ./data/20210805/influxOP_RELIANCE21AUG_20210805.csv
2021-08-26 20:00:00
loading data from ./data/20210806/influxOP_RELIANCE21AUG_20210806.csv
2021-08-26 20:00:00


In [4]:
strikes_arr = oc1.df.strike.values

In [5]:
res1 = oc1.ivs_prices_greeks(strikes_arr)
res1

,strike,futPrice,moneyness,instrument,time_to_expiry,fit_iv,delta,gamma,theta,vega,fit_price,moneyness,greek_0,greek_1,greek_2
0,176000,213435.0,-0.192848,p,0.058076,0.398424,-0.019866,0.000048,-172929.793211,50413.585983,154.300444,-0.192848,0.037190,-0.192848,1.0
1,178000,213435.0,-0.181549,p,0.058076,0.375816,-0.020189,0.000051,-163063.448360,50396.973501,147.940432,-0.181549,0.032960,-0.181549,1.0
2,180000,213435.0,-0.170375,p,0.058076,0.354812,-0.020910,0.000054,-153836.766541,50359.930604,145.057466,-0.170375,0.029028,-0.170375,1.0
3,182000,213435.0,-0.159326,p,0.058076,0.335361,-0.022120,0.000057,-145223.739319,50297.685184,145.872043,-0.159326,0.025385,-0.159326,1.0
4,184000,213435.0,-0.148397,p,0.058076,0.317416,-0.023951,0.000060,-137195.188483,50203.484015,150.853890,-0.148397,0.022022,-0.148397,1.0
5,186000,213435.0,-0.137586,p,0.058076,0.300929,-0.026590,0.000063,-129717.463088,50067.763475,160.812590,-0.137586,0.018930,-0.137586,1.0
6,188000,213435.0,-0.126890,p,0.058076,0.285856,-0.030298,0.000066,-122750.684196,49877.007112,177.033056,-0.126890,0.016101,-0.126890,1.0
7,190000,213435.0,-0.116308,p,0.058076,0.272154,-0.035446,0.000069,-116246.454679,49612.222187,201.474993,-0.116308,0.013528,-0.116308,1.0
8,192000,213435.0,-0.105837,p,0.058076,0.259782,-0.042547,0.000072,-110145.039463,49247.019118,237.059877,-0.105837,0.011201,-0.105837,1.0
9,194000,213435.0,-0.095474,p,0.058076,0.248699,-0.052299,0.000074,-104372.236814,48745.418724,288.069605,-0.095474,0.009115,-0.095474,1.0


In [6]:
res2 = oc2.ivs_prices_greeks(strikes_arr)
res2

,strike,futPrice,moneyness,instrument,time_to_expiry,fit_iv,delta,gamma,theta,vega,fit_price,moneyness,greek_0,greek_1,greek_2
0,176000,213200.0,-0.191747,p,0.055878,0.391536,-0.017087,0.000050,-173549.435978,49536.243319,125.231654,-0.191747,0.036767,-0.191747,1.0
1,178000,213200.0,-0.180447,p,0.055878,0.371804,-0.017994,0.000052,-164651.363010,49490.527120,125.843593,-0.180447,0.032561,-0.180447,1.0
2,180000,213200.0,-0.169274,p,0.055878,0.353484,-0.019339,0.000055,-156323.723390,49422.728514,129.570627,-0.169274,0.028654,-0.169274,1.0
3,182000,213200.0,-0.158224,p,0.055878,0.336530,-0.021242,0.000058,-148537.139901,49326.825477,136.949111,-0.158224,0.025035,-0.158224,1.0
4,184000,213200.0,-0.147295,p,0.055878,0.320899,-0.023869,0.000060,-141258.154611,49194.456057,148.814482,-0.147295,0.021696,-0.147295,1.0
5,186000,213200.0,-0.136484,p,0.055878,0.306552,-0.027448,0.000063,-134447.700472,49014.051069,166.416617,-0.136484,0.018628,-0.136484,1.0
6,188000,213200.0,-0.125789,p,0.055878,0.293448,-0.032296,0.000065,-128059.167048,48769.713343,191.584476,-0.125789,0.015823,-0.125789,1.0
7,190000,213200.0,-0.115207,p,0.055878,0.281551,-0.038842,0.000068,-122036.094110,48439.850048,226.955615,-0.115207,0.013273,-0.115207,1.0
8,192000,213200.0,-0.104735,p,0.055878,0.270823,-0.047655,0.000070,-116309.689957,47995.658855,276.285440,-0.104735,0.010969,-0.104735,1.0
9,194000,213200.0,-0.094373,p,0.055878,0.261229,-0.059480,0.000071,-110796.659275,47399.744207,344.843660,-0.094373,0.008906,-0.094373,1.0


In [7]:
df = pd.DataFrame(strikes_arr, columns=['strike'])

df['delta_price'] = res1['delta'] * (oc2.future_price() - oc1.future_price())
df['gamma_price'] = (res1['gamma'] / 2.0) * ((oc2.future_price() - oc1.future_price())**2)
df['theta_price'] = res1['theta'] * (oc1.time_to_expiry() - oc2.time_to_expiry())

for i in range(len(oc1.iv_parameters())):
    df[f'vol_{i}_price'] = res1['vega'] * res1[f'greek_{i}'] * (oc2.iv_parameters()[i] - oc1.iv_parameters()[i])
     
    
df['estimated_price_diff'] = df['delta_price'] + df['gamma_price'] + df['theta_price']
for i in range(len(oc1.iv_parameters())):
    df['estimated_price_diff'] += df[f'vol_{i}_price']
df['actual_price_diff'] = res2['fit_price'] - res1['fit_price']
df['pricing_error'] = df['actual_price_diff'] - df['estimated_price_diff']

df

,strike,delta_price,gamma_price,theta_price,vol_0_price,vol_1_price,vol_2_price,estimated_price_diff,actual_price_diff,pricing_error
0,176000,4.668601,1.320636,-380.011265,-1201.281167,-41.256958,995.520679,-621.039474,-29.068790,591.970684
1,178000,4.744500,1.399620,-358.330084,-1064.281266,-38.826792,995.192631,-460.101390,-22.096839,438.004551
2,180000,4.913743,1.481385,-338.054555,-936.622525,-36.410440,994.461142,-310.231250,-15.486839,294.744411
3,182000,5.198132,1.565367,-319.127525,-818.059052,-34.006928,993.231977,-171.198028,-8.922932,162.275096
4,184000,5.628522,1.650770,-301.484875,-708.348367,-31.614873,991.371780,-42.797043,-2.039408,40.757635
5,186000,6.248607,1.736504,-285.052658,-607.253203,-29.232440,988.691697,75.138506,5.604026,-69.534479
6,188000,7.120141,1.821104,-269.743240,-514.544571,-26.857325,984.924817,182.720926,14.551420,-168.169506
7,190000,8.329900,1.902635,-255.450270,-430.006571,-24.486852,979.696091,279.984934,25.480622,-254.504311
8,192000,9.998453,1.978576,-242.042479,-353.443469,-22.118263,972.484400,366.857218,39.225563,-327.631655
9,194000,12.290183,2.045693,-229.356810,-284.689152,-19.749378,962.579261,443.119796,56.774056,-386.345740


In [132]:
oc1.iv_parameters()

array([5.38081369, 0.01378637, 0.20096769])

In [131]:
oc2.iv_parameters()

array([4.74009814, 0.01802996, 0.22071476])

In [133]:
36438.189370 * 0.02

728.7637874000001